# Evaluation
Nous allons maintenant évaluer le résultat du parseur et simuler un accord inter-annotateur.

In [ ]:
PATH_TREEBANK = "../data/sud_naija-NSC.with_prediction.conllu"

In [ ]:
# le fichier est t-il bien présent ?
try:
    with open(PATH_TREEBANK, "r") as f:
        print("Le fichier est présent au chemin : {}".format(PATH_TREEBANK))
        pass
except FileNotFoundError:
    print("Le fichier n'est PAS , revoyez le chemin : {}".format(PATH_TREEBANK))
    exit()

In [ ]:
# On peut commencer à parser le fichier avec la librairie conllup
from conllup.conllup import readConlluFile 
sentences = readConlluFile(PATH_TREEBANK)

if len(sentences) == 0:
    raise ValueError("Le fichier est vide !")
else :
    print("Le fichier contient {} phrases".format(len(sentences)))

In [ ]:
# Calculons le coefficient de Kappa-Cohen pour les catégories syntaxiques
from conllup.conllup import readConlluFile

# On récupère les catégories syntaxiques
gold_UPOS = []
pred_UPOS = []
for sentence in sentences:
    for token in sentence['treeJson']['nodesJson'].values():
        gold_UPOS.append(token['UPOS'])
        pred_UPOS.append(token['MISC']['upos_pred'])

if len(gold_UPOS) != len(pred_UPOS):
    raise ValueError("Les deux listes ne sont pas de la même taille !")
print("Les deux listes sont de la même taille : {} éléments".format(len(gold_UPOS)))


## Explication du Kappa de Cohen

La formule du score de Cohen Kappa est conçue pour mesurer l'accord entre deux évaluateurs en prenant en compte la possibilité d'un accord dû au hasard. Voici la formule :

$$
\kappa = \frac{P - Q}{1 - P}
$$

Où :

- \(P\) est la proportion observée d'accord entre les annotateurs.
- \(Q\) est la proportion d'accord attendue par hasard.

Pour calculer ces proportions :

1. **Proportion observée d'accord (\(P\))** : C'est la proportion de fois où les deux annotateurs sont d'accord. Vous la calculez en comptant le nombre de fois où les deux annotateurs ont donné la même étiquette, puis en divisant ce nombre par le nombre total d'éléments annotés.

2. **Proportion d'accord attendue par hasard (\(Q\))** : C'est un peu plus compliqué. Pour chaque catégorie d'étiquette, vous calculez la probabilité que les deux annotateurs choisissent cette catégorie par hasard. Cela se fait en calculant la proportion de fois où chaque annotateur a utilisé chaque catégorie, puis en multipliant ces proportions pour chaque catégorie et en les additionnant. Formellement, si vous avez \( n \) catégories, et que la proportion de fois où l'annotateur 1 a choisi la catégorie \( i \) est \( p_{1i} \) et la proportion pour l'annotateur 2 est \( p_{2i} \), alors \( P_e = \sum_{i=1}^{n} (p_{1i} \times p_{2i}) \).


Le score de Kappa peut être interprété comme suit :

- Un score de 1 indique un accord parfait.
- Un score de 0 indique que l'accord est exactement celui attendu par hasard.
- Un score négatif indique un désaccord.

Cohen Kappa est particulièrement utile car il tient compte de l'accord dû au hasard, ce qui le rend plus robuste que de simplement calculer le pourcentage d'accord direct.


In [ ]:
# On calcule le coefficient de Kappa-Cohen à la main

## On calcule la proportion observée d'accord (probabilité d'accord total)

number_of_agreements = 0
total_number_of_tokens = 0

for (gold, pred) in zip(gold_UPOS, pred_UPOS):
    if gold == pred:
        number_of_agreements += 1
    total_number_of_tokens += 1

p = number_of_agreements / total_number_of_tokens

print("La probabilité d'accord par catégorie est de : {}".format(p))

## maintenant, on calcule la proportion attendue d'accord (probabilité d'accord par chance) Q
number_of_category = 0

sum_of_products = 0
for category in set(gold_UPOS):

    probability_in_gold = gold_UPOS.count(category) / len(gold_UPOS)
    probability_in_pred = pred_UPOS.count(category) / len(pred_UPOS)
    
    print(category, probability_in_gold, probability_in_pred)
    sum_of_products += probability_in_gold * probability_in_pred
    number_of_category += 1

q = sum_of_products / number_of_category
print("La probabilité d'accord par chance (par categorie) est de : {}".format(q))

# On calcule le coefficient de Kappa-Cohen
k_upos = (p - q) / (1 - q)
print("Le coefficient de Kappa-Cohen est de : {}".format(k_upos))


In [ ]:
# Faisons une fonction de ce code

def kappa_cohen(gold_cat, pred_cat):
    """ 
    Calcule le coefficient de Kappa-Cohen pour deux listes de valeurs 
    """     
    number_of_agreements = 0
    total_number_of_tokens = 0

    for (gold, pred) in zip(gold_cat, pred_cat):
        if gold == pred:
            number_of_agreements += 1
        total_number_of_tokens += 1

    p = number_of_agreements / total_number_of_tokens

    print("La probabilité d'accord par catégorie est de : {}".format(p))

    ## maintenant, on calcule la proportion attendue d'accord (probabilité d'accord par chance) Q
    number_of_category = 0

    sum_of_products = 0
    for category in set(gold_cat):
        probability_in_gold = gold_cat.count(category) / len(gold_cat)
        probability_in_pred = pred_cat.count(category) / len(pred_cat)
        sum_of_products += probability_in_gold * probability_in_pred
        number_of_category += 1

    q = sum_of_products / number_of_category
    print("La probabilité d'accord par chance (par categorie) est de : {}".format(q))

    # On calcule le coefficient de Kappa-Cohen
    k = (p - q) / (1 - q)
    print("Le coefficient de Kappa-Cohen est de : {}".format(k))
    return k

# On vérifie que la fonction donne bien le même résultat que le code précédent
assert kappa_cohen(gold_UPOS, pred_UPOS) == k_upos

In [ ]:
# Faisons donc de même pour les catégories de dépendances
gold_DEPREL = []
pred_DEPREL = []
for sentence in sentences:
    for token in sentence['treeJson']['nodesJson'].values():
        gold_DEPREL.append(token['DEPREL'])
        pred_DEPREL.append(token['MISC']['deprel_pred'])

if len(gold_DEPREL) != len(pred_DEPREL):
    raise ValueError("Les deux listes ne sont pas de la même taille !")
print("Les deux listes sont de la même taille : {} éléments".format(len(gold_DEPREL)))



In [ ]:
# On calcule le coefficient de Kappa-Cohen
k_deprel = kappa_cohen(gold_DEPREL, pred_DEPREL)

L'étiquette de relation syntaxique semble plus compliquée à prédire que l'étiquette de catégorie du discours.

NB : On ne peut pas appliquer ce coefficient Kappa-Cohen à la prédiction des gouverneur puisque cette prédiction n'est pas un problème de classification. 

In [ ]:
# Calculons maintenant les trois scores LAS, UAS et LAS
# On commence par calculer le nombre de tokens correctement étiquetés
correctly_labeled_tokens = 0
correctly_attached_tokens = 0
correctly_labeled_and_attached_tokens = 0
correct_UPOSs = 0
total_number_of_tokens = 0

for sentence in sentences:
    for token in sentence['treeJson']['nodesJson'].values():
        head_token_ID = str(token['HEAD'])
        deprel = token['DEPREL']
        upos = token['UPOS']

        head_pred = token['MISC']['head_pred']
        deprel_pred = token['MISC']['deprel_pred']
        upos_pred = token['MISC']['upos_pred']

        if head_token_ID == head_pred:
            correctly_attached_tokens += 1

        if deprel == deprel_pred:
            correctly_labeled_tokens += 1
        
        if head_token_ID == head_pred and deprel == deprel_pred:
            correctly_labeled_and_attached_tokens += 1

        if upos == upos_pred:
            correct_UPOSs += 1

        total_number_of_tokens += 1

LAS = correctly_labeled_and_attached_tokens / total_number_of_tokens
LUS = correctly_labeled_tokens / total_number_of_tokens
UAS = correctly_attached_tokens / total_number_of_tokens
UPOS = correct_UPOSs / total_number_of_tokens

print("Le score LAS est de : {}".format(LAS))
print("Le score LUS est de : {}".format(LUS))
print("Le score UAS est de : {}".format(UAS))
print("Le score UPOS est de : {}".format(UPOS))



In [ ]:
# Combien de phrases ont au moins un token mal étiqueté (pour chaque caractéristique) ?

In [ ]:
# Il faudrait maintenant que nous recalculions toutes ces mesures sans les ponctuations